In [1]:
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [32]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# K.set_image_data_format('channels_first')

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('K.image_data_format()', K.image_data_format())
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('input_shape:', input_shape)

K.image_data_format() channels_first
x_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples
input_shape: (1, 28, 28)


In [33]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('num_classes:', num_classes)

num_classes: 10


In [34]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 26, 26)        320       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 64, 24, 24)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 12, 12)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 64, 12, 12)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
__________

In [37]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 419s 7ms/step - loss: 0.2540 - val_loss: 0.0522
Epoch 2/12
60000/60000 [==============================] - 447s 7ms/step - loss: 0.0873 - val_loss: 0.0402
Epoch 3/12
60000/60000 [==============================] - 424s 7ms/step - loss: 0.0658 - val_loss: 0.0322
Epoch 4/12
60000/60000 [==============================] - 418s 7ms/step - loss: 0.0529 - val_loss: 0.0318
Epoch 5/12
60000/60000 [==============================] - 584s 10ms/step - loss: 0.0461 - val_loss: 0.0298
Epoch 6/12
60000/60000 [==============================] - 411s 7ms/step - loss: 0.0406 - val_loss: 0.0321
Epoch 7/12
60000/60000 [==============================] - 410s 7ms/step - loss: 0.0336 - val_loss: 0.0291
Epoch 8/12
60000/60000 [==============================] - 414s 7ms/step - loss: 0.0294 - val_loss: 0.0288
Epoch 9/12
60000/60000 [==============================] - 414s 7ms/step - loss: 0.0290 - val_loss: 0.

IndexError: invalid index to scalar variable.

In [39]:
print('score:', score)

score: 0.02777003308636481


In [40]:
model.save('model-mnist.h5')